In [23]:
import pandas as pd
import json
import re
from string import punctuation
import numpy as np
import metadata_funs

In [54]:
ers_path = 'FY2014-19 Datasets for ERS Publications.xlsx'
ers_df_list = []
ers_xls = pd.ExcelFile(ers_path)
sheet_names = ers_xls.sheet_names

In [55]:
sheet_names_lim = ['Census Surveys', 'CES', 'CPS-FSS', 'Cost Est. Foodborne Illnesses'
                   , 'FDA-FSIS-AMS', 'FADS-LAFA', 'FARA', 'FICRCD-CSFII', 'FNS', 'FoodAPS'
                   ,'IRI', 'NHANES', 'Nielsen', 'Qtrly FAFH', 'SNAP Admin', 'TD Linx', 'SCH MEALS DATA']

In [57]:
for n in sheet_names_lim:
# for n in ['FY14', 'FY15']:
    ers_pubs = pd.read_excel(ers_path, sheet_name = n,header = 1)
    this_df = pd.DataFrame({'titles':ers_pubs['Titles'],'sheet_name':np.repeat(n,len(ers_pubs['Datasets']))
          ,'datasets':ers_pubs['Datasets']})
    ers_df_list.append(this_df)

In [58]:
ers_df = pd.concat(ers_df_list)
ers_df = ers_df.drop_duplicates()
ers_df = ers_df.dropna(subset=['datasets'])

In [59]:
ers_df['pub_uid'] = metadata_funs.get_hash(ers_df['titles'])
ers_df.head()

,titles,sheet_name,datasets,pub_uid
0,Participation in SNAP and Unemployment Insuran...,Census Surveys,▪CPS ▪ASEC,6fe5f86987168eae17ca
1,"Measuring Access to Healthful, Affordable Food...",Census Surveys,▪2010 Census TIGER/Line® Shapefiles ▪2010 Cens...,6fe5f86987168eae17ca
2,Annual and Monthly SNAP Participation Rates,Census Surveys,▪SIPP and ACS,6fe5f86987168eae17ca
3,Food Choices and Store Proximity,Census Surveys,▪U.S. Census Bureau (2010 Census Tract),6fe5f86987168eae17ca
4,Improving the Assessment of SNAP Targeting Usi...,Census Surveys,▪American Community Survey (ACS),6fe5f86987168eae17ca


In [60]:
split_df = pd.concat([pd.Series(row['pub_uid'], row['datasets'].split('▪'))              
                    for _, row in ers_df.iterrows()]).reset_index()

split_df = split_df.rename(index=str, columns={"index": "datasets", 0: "pub_uid"})
ers_df_full = pd.merge(ers_df,split_df,how = "inner", on = "pub_uid")
ers_df_full_lim = ers_df_full[['titles','sheet_name','pub_uid','datasets_y']]
# ers_df_full_lim.head()

In [61]:
ers_df_clean = ers_df_full_lim[ers_df_full_lim['datasets_y'].astype(bool)      ]

In [62]:
ers_df_clean.head()
ers_df_clean.sheet_name.unique()



array(['Census Surveys', 'CES', 'CPS-FSS',
       'Cost Est. Foodborne Illnesses', 'FDA-FSIS-AMS', 'FADS-LAFA',
       'FARA', 'FICRCD-CSFII', 'FNS', 'FoodAPS', 'IRI', 'NHANES',
       'Nielsen', 'SNAP Admin', 'TD Linx', 'SCH MEALS DATA'], dtype=object)

In [67]:
def data_creator(row):
    if row['sheet_name'] =='Census Surveys':
        val = "U.S. Census Bureau"
    elif row['sheet_name'] =='CES':
        val = "Bureau of Labor Statistics"
    elif row['sheet_name'] =='CPS-FSS':
        val = ["Bureau of Labor Statistics","U.S. Census Bureau","U.S. Department of Agriculture"]
    elif row['sheet_name'] in ['SCH MEALS DATA','SNAP Admin','Cost Est. Foodborne Illnesses','FDA-FSIS-AMS','FADS-LAFA','FARA','FICRCD-CSFII','FNS','FoodAPS']:
        val = "U.S. Department of Agriculture"
    elif row['sheet_name'] in ['IRI','Nielsen','TD Linx']:
        val = "Nielsen"
    else:
        val = row['sheet_name']
    return val

# def data_provider(row):
#     if row['sheet_name'] =='Census Surveys':
#         val = "U.S. Census Bureau"
#     elif row['A'] > row['B']:
#         val = 1
#     else:
#         val = -1
#     return val

In [68]:
# data_provider
ers_df_clean['C'] = ers_df_clean.apply(data_creator, axis=1)


/Users/sophierand/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [69]:
ers_df_clean

,titles,sheet_name,pub_uid,datasets_y,C
1,Participation in SNAP and Unemployment Insuran...,Census Surveys,6fe5f86987168eae17ca,CPS,U.S. Census Bureau
2,Participation in SNAP and Unemployment Insuran...,Census Surveys,6fe5f86987168eae17ca,ASEC,U.S. Census Bureau
4,Participation in SNAP and Unemployment Insuran...,Census Surveys,6fe5f86987168eae17ca,2010 Census TIGER/Line® Shapefiles,U.S. Census Bureau
5,Participation in SNAP and Unemployment Insuran...,Census Surveys,6fe5f86987168eae17ca,2010 Census of Population,U.S. Census Bureau
6,Participation in SNAP and Unemployment Insuran...,Census Surveys,6fe5f86987168eae17ca,"American Community Survey, 2006-2010 Summary",U.S. Census Bureau
8,Participation in SNAP and Unemployment Insuran...,Census Surveys,6fe5f86987168eae17ca,SIPP and ACS,U.S. Census Bureau
10,Participation in SNAP and Unemployment Insuran...,Census Surveys,6fe5f86987168eae17ca,U.S. Census Bureau (2010 Census Tract),U.S. Census Bureau
12,Participation in SNAP and Unemployment Insuran...,Census Surveys,6fe5f86987168eae17ca,American Community Survey (ACS),U.S. Census Bureau
14,Participation in SNAP and Unemployment Insuran...,Census Surveys,6fe5f86987168eae17ca,Survey of Income and Program Participation (SIPP),U.S. Census Bureau
16,Participation in SNAP and Unemployment Insuran...,Census Surveys,6fe5f86987168eae17ca,U.S. Census Bureau (School Food Authority Char...,U.S. Census Bureau


In [10]:
ers_datasets = ers_df.datasets.unique().tolist()
# ers_datasets[0:1]

In [11]:
flatten = lambda l: [item for sublist in l for item in sublist]

In [21]:
a = list(set([b.strip() for b in flatten([e.split('▪') for e in  ers_datasets]) if b not in ['',' ']]))
b = [{'dataset_name':b} for b in a]

# json.dumps(a,'usda_datasets_dirty.json',indent = 2)
with open('usda_datasets_dirty.json', 'w') as outfile:
    json.dump(b, outfile,indent = 2)

In [ ]:
flatten = lambda l: [item for sublist in l for item in sublist]

In [ ]:
ers_datasets = ers_pubs['Datasets'].unique().tolist()
# [e.replace()ers_datasets
# [re.sub('[^A-Za-z0-9]+', ' ', e).strip() for e in ers_datasets]
# [re.sub('▪', ',', e).strip(punctuation) for e in ers_datasets][]
ers_datasets_clean = [f.replace('\n',' ').strip() for f in flatten([e.split('▪') for e in ers_datasets]) if f not in ['',' ']]
ers_datasets_clean



In [ ]:
from string import punctuation
s = '.$ABC-799-99,#'

print(s.strip(punctuation))